# Temporal LoRA for Dynamic Sentence Embeddings - Demo

This notebook demonstrates the complete pipeline on a small dataset suitable for Google Colab (T4 GPU).

**Pipeline Steps:**
1. Environment setup
2. Data preparation (6000 samples per time bucket)
3. Train LoRA adapters (2 epochs, rank=16)
4. Build indexes and evaluate
5. Visualize results

**Estimated Runtime:** ~15-20 minutes on T4 GPU

## Setup

In [ ]:
# Install dependencies (uncomment if running on Colab)
# !pip install -q torch transformers sentence-transformers peft datasets faiss-cpu pandas numpy matplotlib seaborn umap-learn typer

In [ ]:
# Clone repository (uncomment if running on Colab)
# !git clone https://github.com/yourusername/DynamicEmbeddings.git
# %cd DynamicEmbeddings

In [ ]:
import sys
from pathlib import Path

# Add project to path
project_root = Path.cwd()
if str(project_root / "src") not in sys.path:
    sys.path.insert(0, str(project_root / "src"))

print(f"Project root: {project_root}")

In [ ]:
# Check GPU availability
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
    print(f"Device count: {torch.cuda.device_count()}")

## Step 1: Environment Dump

In [ ]:
# Capture environment for reproducibility
!python -m temporal_lora.cli env-dump

## Step 2: Prepare Data

Download and preprocess arXiv CS/ML abstracts into time buckets (≤2018, 2019-2024).
Using 6000 samples per bucket for quick demo.

In [ ]:
!python -m temporal_lora.cli prepare-data --max-per-bucket 6000

## Step 3: Train LoRA Adapters

Train time-bucket LoRA adapters with:
- Rank: 16
- Epochs: 2
- Cross-period negatives: enabled

In [ ]:
!python -m temporal_lora.cli train-adapters --epochs 2 --lora-r 16 --cross-period-negatives

## Step 4: Build Indexes & Evaluate

Build FAISS indexes and evaluate both baseline and LoRA systems.

In [ ]:
# Build baseline indexes
!python -m temporal_lora.cli build-indexes --baseline

In [ ]:
# Build LoRA indexes
!python -m temporal_lora.cli build-indexes --lora

In [ ]:
# Evaluate baseline
!python -m temporal_lora.cli evaluate --baseline --mode multi-index --merge softmax

In [ ]:
# Evaluate LoRA
!python -m temporal_lora.cli evaluate --lora --mode multi-index --merge softmax

## Step 5: Visualize Results

In [ ]:
# Generate heatmaps and UMAP
!python -m temporal_lora.cli visualize

## Results Preview

Let's load and compare the results.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Image

# Load results
baseline_df = pd.read_csv("deliverables/results/baseline_results.csv", index_col=0)
lora_df = pd.read_csv("deliverables/results/lora_results.csv", index_col=0)

print("\n" + "="*60)
print("BASELINE RESULTS")
print("="*60)
display(baseline_df)

print("\n" + "="*60)
print("LORA RESULTS")
print("="*60)
display(lora_df)

In [ ]:
# Compute improvements
print("\n" + "="*60)
print("IMPROVEMENTS (Δ = LoRA - Baseline)")
print("="*60)

delta_df = lora_df - baseline_df
display(delta_df)

# Summary statistics
print("\nMean Improvements:")
for col in delta_df.columns:
    mean_delta = delta_df[col].mean()
    print(f"  {col.upper()}: {mean_delta:+.4f}")

## Visualization: Comparison Heatmaps

In [ ]:
# Display NDCG@10 comparison heatmap
heatmap_path = "deliverables/figures/comparison_heatmaps_ndcg@10.png"
if Path(heatmap_path).exists():
    display(Image(filename=heatmap_path))
else:
    print(f"Heatmap not found: {heatmap_path}")

In [ ]:
# Display UMAP projection
umap_path = "deliverables/figures/umap_embeddings.png"
if Path(umap_path).exists():
    display(Image(filename=umap_path))
else:
    print(f"UMAP not found: {umap_path}")

## Export Deliverables

Consolidate all results into a single deliverables package.

In [ ]:
!python -m temporal_lora.cli export-deliverables

## Summary

✅ **Demo Complete!**

The pipeline has successfully:
1. Prepared time-bucketed data
2. Trained LoRA adapters for each time period
3. Built FAISS indexes for retrieval
4. Evaluated performance with multiple metrics
5. Generated visualizations
6. Exported deliverables with reproducibility info

**Key Findings:**
- LoRA adapters enable **time-aware embeddings** without retraining the base model
- Performance improvements are visible in the **Δ (improvement) heatmap**
- Cross-period queries benefit from multi-index retrieval with adaptive merging

**Next Steps:**
- Run ablations to optimize hyperparameters
- Analyze term drift trajectories
- Scale to larger datasets